## Read the wrfout* files

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import proplot as plot

from tropCalc import tropCalc
from metpy.units import units
from metpy.calc import pressure_to_height_std
from wrf import getvar, interplevel, ALL_TIMES, CoordPair, vertcross, to_np

In [ ]:
def get_var(wrf_in, varname, units=None):
    '''Get variables from wrf file'''
    try:
        var = getvar(wrf_in, varname, timeidx=ALL_TIMES,
                     method='cat', units=units)
    except:
        # in case the var doesn't have "units" arg
        var = getvar(wrf_in, varname, timeidx=ALL_TIMES,
                     method='cat')

    return var

In [ ]:
lnox_dir = '../data/wrfchem/wrfout/20190725/lnox_500/'
nolnox_dir = '../data/wrfchem/wrfout/20190725/nolnox/'

lnox_file = sorted(glob(lnox_dir+'wrfout_d03_2019-07-25_0[3-6]*'))
nolnox_file = sorted(glob(nolnox_dir+'wrfout_d03_2019-07-25_0[3-6]*'))

wrf_lnox = [Dataset(f) for f in lnox_file]
wrf_nolnox = [Dataset(f) for f in nolnox_file]

In [ ]:
rh = get_var(wrf_lnox, 'rh')

In [ ]:
qv = get_var(wrf_lnox, 'QVAPOR')
qcld = get_var(wrf_lnox, 'QCLOUD')
qgraup = get_var(wrf_lnox, 'QGRAUP')
qice = get_var(wrf_lnox, 'QICE')

In [ ]:
T = get_var(wrf_lnox, 'T')
P = get_var(wrf_lnox, 'pressure')

t = get_var(wrf_lnox, 'times')
z = get_var(wrf_lnox, 'z', units='km')
zstag = get_var(wrf_lnox, 'zstag', units='km')
w = get_var(wrf_lnox, 'wa')

o3 =  get_var(wrf_lnox, 'o3')
o3_nolnox =  get_var(wrf_nolnox, 'o3')

advh_o3 = get_var(wrf_lnox, 'advh_o3')
advh_o3_nolnox = get_var(wrf_nolnox, 'advh_o3')

advz_o3 = get_var(wrf_lnox, 'advz_o3')
advz_o3_nolnox = get_var(wrf_nolnox, 'advz_o3')

chem_o3 = get_var(wrf_lnox, 'chem_o3')
chem_o3_nolnox = get_var(wrf_nolnox, 'chem_o3')

In [ ]:
no2 =  get_var(wrf_lnox, 'no2')
no2_nolnox =  get_var(wrf_nolnox, 'no2')

## Plot the o3 profiles

### Subset to region

In [ ]:
# subset to the interested region
lon = o3.XLONG
lat = o3.XLAT

crop_region = [118.95, 119.10, 31.93, 31.97]

# subset = (lon>=118.98) & (lon<=119.13) & (lat>=31.9) & (lat<=32)
subset = (lon>=crop_region[0]) & (lon<=crop_region[1]) & (lat>=crop_region[2]) & (lat<=crop_region[3])

# calculate the mean profile
o3_region = o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
o3_region_nolnox = o3_nolnox.where(subset, drop=True).mean(dim=['south_north', 'west_east'])

chem_o3_region = chem_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
chem_o3_region_nolnox = chem_o3_nolnox.where(subset, drop=True).mean(dim=['south_north', 'west_east'])

z_region = z.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
zstag_region = zstag.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
w_region = w.where(subset, drop=True).mean(dim=['south_north', 'west_east'])

display(o3_region)

In [ ]:
no2_region = no2.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
no2_region_nolnox = no2_nolnox.where(subset, drop=True).mean(dim=['south_north', 'west_east'])


### Qickview

In [ ]:
p_region = P.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
t_region = T.where(subset, drop=True).mean(dim=['south_north', 'west_east'])

lapseC = 2*units.kelvin/units.km
height = False

tropo = tropCalc(p_region.isel(Time=0).values*units.hPa,
                 t_region.isel(Time=0).values*units.K,
                 lapseC=lapseC,
                 height=height,
                 method="wmo")
pressure_to_height_std(tropo)

In [ ]:
o3_region.isel(Time=slice(0, 6)).plot(y='bottom_top', hue='Time')
plt.ylim(0, 40)
plt.xlim(0, 0.2)

In [ ]:
o3_region.isel(Time=slice(6, 12)).plot(y='bottom_top', hue='Time')
plt.ylim(0, 40)
plt.xlim(0, 0.2)

In [ ]:
o3_region.isel(Time=slice(12, 18)).plot(y='bottom_top', hue='Time')
plt.ylim(0, 40)
plt.xlim(0, 0.2)

The O3 profile between layer 20 and 25:

- constant from 03:20 to 04:00

- decreasing from 04:10 to 04:50

- increaseing from 04:50 to 06:10

In [ ]:
(o3_region - o3_region_nolnox).plot()

It seems lightning NO has different effects on two layers:

increasing O3 (10 -- 20 levs) and decreasing O3 (20 -- 30 levs).

How about the chem_tend?

### Pick specific time based on the convection

In [ ]:
# see comp_wrf_radar.ipynb
subset_t = pd.to_datetime(['2019-07-25 03:20', '2019-07-25 03:40',
                           '2019-07-25 04:20', '2019-07-25 04:40',
                           '2019-07-25 05:20', '2019-07-25 05:40'])

o3_profile = o3_region.sel(Time=subset_t)
o3_profile_nolnox = o3_region_nolnox.sel(Time=subset_t)
chem_o3_profile = chem_o3_region.sel(Time=subset_t)
chem_o3_profile_nolnox = chem_o3_region_nolnox.sel(Time=subset_t)

# do the same for altitude and w
z_profile = z_region.sel(Time=subset_t)
zstag_profile = zstag_region.sel(Time=subset_t)
w_profile = w_region.sel(Time=subset_t)

In [ ]:
rh_region = (rh.where(subset, drop=True).mean(dim=['south_north', 'west_east'])).assign_coords(bottom_top=z_profile.isel(Time=0).values)
qv_region = (qv.where(subset, drop=True).mean(dim=['south_north', 'west_east'])*1e3).assign_coords(bottom_top=z_profile.isel(Time=0).values)
qall_region = ((qcld+qice+qgraup).where(subset, drop=True).mean(dim=['south_north', 'west_east'])*1e3).assign_coords(bottom_top=z_profile.isel(Time=0).values)
# fig, axs = plot.subplots(ncols=2, nrows=2)
fig, axs = plot.subplots()

import matplotlib
cmap = matplotlib.cm.get_cmap('Spectral')

lines = []
num_line = 15 # qv_region.sizes['Time']
# for index in range(qv_region.sizes['Time']):
for index in range(num_line):
    l = axs.plot(
                rh_region.isel(Time=index),
#                 (qall_region+qv_region).isel(Time=index),
             z_profile.isel(Time=0),
             color = cmap(index/num_line),
#              label = (qv_region+qcld).isel(Time=index).Time.dt.strftime('%H:%M').values
             label = qv_region.isel(Time=index).Time.dt.strftime('%H:%M').values
            )
    lines.append(l)
axs.legend(lines, loc='r')
axs.format(
            xlim=(20, 60),
           ylim=(12, 17),
           xlabel='RH (%)',
           ylabel='Altitude (km)')

No obvious signal of transported stratospheric air ... The higher O3 should be the UT O3 at the higher level.

In [ ]:
fig, axs = plot.subplots(axwidth=5)
lines = []
num_line = 15 # qv_region.sizes['Time']
# for index in range(qv_region.sizes['Time']):
for index in range(num_line):
    l = axs.plot(
                w_region.isel(Time=index),
#                 (qall_region+qv_region).isel(Time=index),
             z_profile.isel(Time=0),
             color = cmap(index/num_line),
#              label = (qv_region+qcld).isel(Time=index).Time.dt.strftime('%H:%M').values
             label = qv_region.isel(Time=index).Time.dt.strftime('%H:%M').values
            )
    lines.append(l)
axs.legend(lines, loc='r')
axs.format(
            xlim=(-1, 1),
           ylim=(8, 15),
           xlabel='w (m/s)',
           ylabel='Altitude (km)')

In [ ]:
fig, axs = plot.subplots()
time = '2019-07-25 04:40'
axs.plot(w_region.sel(Time=time), z_profile.isel(Time=0))
twiny = axs.twiny()
twiny.plot(o3_region.sel(Time=time), z_profile.isel(Time=0), color='orange')
twiny.format(xlim=(0.06, 0.16))
axs.format(ylim=(8, 15), xlim=(-1.5, 1.5))

In [ ]:
fig, axs = plot.subplots()
time = '2019-07-25 04:40'
axs.plot(advz_o3.sel(Time=time).mean(dim=['south_north', 'west_east']), z_profile.isel(Time=0))
axs.format(ylim=(8, 14), xlim=(-0.02, 0.03))

### Compare the O3 profile in detail

In [ ]:
fig, axs = plot.subplots()

st = 0
et = 2
cmaps = ['blue7', 'green7', 'red7']
legends = []


for index,t in enumerate(range(0, 6, 2)):
    s_profile = o3_profile.isel(Time=t)
    e_profile = o3_profile.isel(Time=t+1)
    s_profile_nolnox = o3_profile_nolnox.isel(Time=t)
    
    s_t = s_profile.Time.dt.strftime('%H:%M').values
    e_t = e_profile.Time.dt.strftime('%H:%M').values
    
    s_legend = axs.plot(s_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='-',
                        color=cmaps[index],
                        label=s_t)
    e_legend = axs.plot(e_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='--',
                        color=cmaps[index],
                        label=e_t)
    s_legend_nolnox = axs.plot(s_profile_nolnox*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='-.',
#                         color=cmaps[index],
                               color='k',
                        label=s_t+' nolnox')
    
    legends.extend([s_legend, e_legend, s_legend_nolnox])

axs.format(xlim=(0, 250),
           ylim=(0, 16),
           xlabel='O$_3$ (ppbv)',
           ylabel='Altitude (km)')
print(legends)
axs.legend(legends, ncols=2, loc='b')

The effect of LNOx isn't obvious on the O3 profile ... It's better to show the difference

In [ ]:
fig, axs = plot.subplots()

st = 0
et = 2
cmaps = ['blue7', 'green7', 'red7']
legends = []

chem_o3_region = chem_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
chem_o3_region_nolnox = chem_o3_nolnox.where(subset, drop=True).mean(dim=['south_north', 'west_east'])


for index,t in enumerate(range(0, 6, 2)):
    s_t = chem_o3_region.sel(Time=subset_t[t]).Time.dt.strftime('%H:%M').values
    s_profile = chem_o3_region.sel(Time=subset_t[t]) - chem_o3_region_nolnox.sel(Time=subset_t[t])
    
    legend = axs.plot(s_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='-',
                        color=cmaps[index],
                        label=s_t)
    
    legends.extend([legend])

axs.format(xlim=(-5, 1),
           ylim=(0, 16),
           xlabel='Difference of O$_3$ (ppbv)',
           ylabel='Altitude (km)')

axs.legend(legends, ncols=2, loc='b')

In [ ]:
fig, axs = plot.subplots()

st = 0
et = 2
cmaps = ['blue7', 'green7', 'red7']
legends = []

chem_o3_region = chem_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])
chem_o3_region_nolnox = chem_o3_nolnox.where(subset, drop=True).mean(dim=['south_north', 'west_east'])


for index,t in enumerate(range(0, 6, 2)):
    s_t = chem_o3_region.sel(Time=subset_t[t]).Time.dt.strftime('%H:%M').values
    s_profile = chem_o3_region.sel(Time=subset_t[t]) - chem_o3_region.isel(Time=0)
    
    legend = axs.plot(s_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='-',
                        color=cmaps[index],
                        label=s_t)
    
    legends.extend([legend])

axs.format(xlim=(-15, 15),
           ylim=(0, 16),
           xlabel='O$_3$ (ppbv)',
           ylabel='Altitude (km)')

axs.legend(legends, ncols=2, loc='b')

## Plot the tendency

### means over region and heights

As shown above, we need to focus on the UT level: 10 -- 14 km. Let's check the mean tendecy:

In [ ]:
# set the levels
bottom = 10 # km
top = 14 # km

advh_o3_mean = advh_o3.where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advh')

advz_o3_mean = advz_o3.where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advz')

total_adv_mean = (advh_o3_mean + advz_o3_mean).rename('advh+advz')

chem_o3_mean = chem_o3.where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('chem')

o3dt_mean = o3_region.where((z_region >= bottom) & (z_region <= top), drop=True)\
                .mean(dim='bottom_top')\
                .rename('dO$_3$dt')

# w_mean = w_region.where((zstag_region >= bottom) & (zstag_region <= top), drop=True)\
#                         .mean(dim='bottom_top_stag')\
#                         .rename('w')
w_mean = w_region.where((z_region >= bottom) & (z_region <= top), drop=True)\
                        .mean(dim='bottom_top')\
                        .rename('w')

In [ ]:
fig, axs = plot.subplots()

legends = []

for var in [advh_o3_mean, advz_o3_mean, chem_o3_mean, o3dt_mean, total_adv_mean]:
    if var.name == 'o3dt':
        color = 'k'
    else:
        color = None
    legend = (var.diff(dim='Time')/
              (var.Time.diff(dim='Time')/np.timedelta64(1, 's')).astype(int)
              *1e6).plot(ax=axs, label=var.name, color=color)
    legends.append(legend)

twin_ax = axs.twinx()
l_w = w_mean.plot(ax=twin_ax, label=w_mean.name,
                  linestyle='--',
                  color='k')
twin_ax.format(ylim=(-0.8, 0.8))

axs.legend(legends, loc='r', ncols=1)
axs.format(ylabel='O$_3$ tendency (pptv/s)')

### Combination

In [ ]:
fig, axs = plot.subplots(ncols=2, share=0)

# subplot1
ax = axs[0]
st = 0
et = 2
cmaps = ['blue7', 'green7', 'red7']
legends = []

for index,t in enumerate(range(0, 6, 2)):
    s_profile = o3_profile.isel(Time=t)
    e_profile = o3_profile.isel(Time=t+1)
    
    s_t = s_profile.Time.dt.strftime('%H:%M').values
    e_t = e_profile.Time.dt.strftime('%H:%M').values
    
    s_legend = ax.plot(s_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='-',
                        color=cmaps[index],
                        label=s_t)
    e_legend = ax.plot(e_profile*1e3, # ppbv
                        z_profile.isel(Time=t),
                        linestyle='--',
                        color=cmaps[index],
                        label=e_t)
    
    legends.extend([s_legend, e_legend])

ax.format(xlim=(0, 200),
           ylim=(0, 16),
           xlabel='O$_3$ (ppbv)',
           ylabel='Altitude (km)')

ax.legend(legends, ncols=1, loc='lr', frame=False)

# subplot2
ax = axs[1]

legends = []

for var in [advh_o3_mean, advz_o3_mean, chem_o3_mean, o3dt_mean, total_adv_mean, w_mean]:
    if var.name == 'advh+advz':
        linestyle = '-'; color = 'blue7'
    elif 'advh' in var.name:
        linestyle = '--'; color = 'blue7'
    elif 'advz' in var.name:
        linestyle = ':'; color = 'blue7'
    elif 'chem' in var.name:
        linestyle = '-.'; color = 'green7'
    elif var.name == 'w':
        linestyle = '-'; color = 'gray7'
    else:
        linestyle = None; color = 'red7'
    
    if var.name == 'w':
        twin_ax = ax.twinx()
        legend = var.plot(ax=twin_ax, label=var.name, color=color, linestyle=linestyle)
        twin_ax.format(ylim=(-0.8, 0.8), ylabel='w (m/s)')
#         twin_ax.tick_params(axis='y', colors='gray7')
#         twin_ax.yaxis.label.set_color('gray7')
        legends.append(*legend)
    else:
        legend = ax.step(var.Time[1:],
                        (var.diff(dim='Time')/(var.Time.diff(dim='Time')/np.timedelta64(1, 's')).astype(int)*1e6),
                        where='pre',
                        color=color,
                        linestyle=linestyle,
                        label=var.name)
#         legend = (var.diff(dim='Time')/
#               (var.Time.diff(dim='Time')/np.timedelta64(1, 's')).astype(int)
#               *1e6).plot(ax=ax, label=var.name, color=color, linestyle=linestyle)
        legends.append(legend)
    
ax.legend(legends, loc='r', ncols=1, frame=False)
ax.format(ylabel='O$_3$ tendency (pptv/s)',
          xlabel='Time (UTC)',
          xlim=(var.Time.values[0], var.Time.values[-1]),
#           ylim=(-22, 22),
          xlocator=('minute', range(0, 60, 30)),
          xminorlocator=('minute', range(0, 60, 10)),
          xformatter='%H:%M',
          title=f'Means ({str(bottom)} ~ {str(top)} km)')

# all subplots
axs.format(grid=False,
           abc=True, abcloc='ul', abcstyle='(a)',
           rowlabels='2019-07-25')

# save fig
# fig.savefig(f'../figures/tendency_o3_2019_mean.pdf')

### Means over region

Back here: Convert ppmv to molec. cm$^{-3}$:

C$_n$ [molec. cm$^{-3}$] = ppp \* P [Pa] \* N$_A$ / (R * T [k]) \*10$^{-6}$

= ppm \* P [hPa] \* N$_A$ / (R * T [k]) \*10$^{-10}$

In [ ]:
advh_o3_h = advh_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
                   .diff(dim='Time').sel(Time=subset_t)

advz_o3_h = advz_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
                   .diff(dim='Time').sel(Time=subset_t)

total_adv_h = (advh_o3_h + advz_o3_h).rename('advh+advz')

chem_o3_h = chem_o3.where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
            .diff(dim='Time').sel(Time=subset_t)

o3dt_h = o3_region.diff(dim='Time').sel(Time=subset_t)

# NA = 6.022e23
# R = 8.314

# advh_o3_h = (advh_o3 * P * NA / (R*T) * 10e-10).where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
#                    .diff(dim='Time').sel(Time=subset_t)

# advz_o3_h = (advz_o3 * P * NA / (R*T) * 10e-10).where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
#                    .diff(dim='Time').sel(Time=subset_t)

# total_adv_h = (advh_o3_h + advz_o3_h).rename('advh+advz')

# chem_o3_h = (chem_o3 * P * NA / (R*T) * 10e-10).where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
#             .diff(dim='Time').sel(Time=subset_t)


# o3dt_h = (o3* P * NA / (R*T) * 10e-10).where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
#             .diff(dim='Time').sel(Time=subset_t)

w_h = w_region.diff(dim='Time').sel(Time=subset_t)

advh_o3_h = advh_o3_h.rename('advh')
advz_o3_h = advz_o3_h.rename('advz')
total_adv_h = total_adv_h.rename('advh+advz')
chem_o3_h = chem_o3_h.rename('chem')
o3dt_h = o3dt_h.rename('dO$_3$dt')
w_h = w_h.rename('w')

In [ ]:
cross_time = [i for i, filename in enumerate(lnox_file) if '2019-07-25_04:40' in filename]
cross_time_init = [i for i, filename in enumerate(lnox_file) if '2019-07-25_03:20' in filename]
ncfile = wrf_lnox[cross_time[0]]
ncfile_init = wrf_lnox[cross_time_init[0]]

# add the simulated o3 crosssection to subplot3
qcld = get_var(ncfile, 'QCLOUD')*1e3
qice = get_var(ncfile, 'QICE')*1e3
qcld += qice
qcld.attrs['units'] = 'g kg-1'

u = get_var(ncfile, 'ua')
v = get_var(ncfile, 'va')
w = get_var(ncfile, 'wa')
T_t = get_var(ncfile, 'tk')
P_t = get_var(ncfile, 'pressure')

start_point = CoordPair(lat=32.15, lon=118.95)
end_point = CoordPair(lat=31.85, lon=119.15)

z_t = getvar(ncfile, 'z', units='km')
o3_t = getvar(ncfile, 'o3')
cross_o3 = vertcross(o3_t, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_qcld = vertcross(qcld, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_u = vertcross(u, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_v = vertcross(v, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_w = vertcross(w, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_t = vertcross(T_t, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)
cross_p = vertcross(P_t, z_t, wrfin=ncfile, start_point=start_point, end_point=end_point, latlon=True, meta=True)

vertical_max = 18
cross_o3 = cross_o3.sel(vertical=slice(0, vertical_max))
cross_qcld = cross_qcld.sel(vertical=slice(0, vertical_max))
cross_u = cross_u.sel(vertical=slice(0, vertical_max))
cross_v = cross_v.sel(vertical=slice(0, vertical_max))
cross_w = cross_w.sel(vertical=slice(0, vertical_max))
cross_t = cross_t.sel(vertical=slice(0, vertical_max))
cross_p = cross_p.sel(vertical=slice(0, vertical_max))

Convert ppmv to molec. cm$^{-3}$:

C$_n$ [molec. cm$^{-3}$] = ppp \* P [Pa] \* N$_A$ / (R * T [k]) \*10$^{-6}$

= ppm \* P [hPa] \* N$_A$ / (R * T [k]) \*10$^{-10}$

In [ ]:
fig, axs = plot.subplots(ncols=3, share=0)#, sharey=2)


axs.format(grid=False,
           #abc=True,
           #abcloc='l',
           #abcstyle='(a)',
           leftlabels=['2019-07-25'],
           leftlabelsize=13,
           textlabelsize=13,
           ticklabelsize=10
          )

# subplot1
ax = axs[0]
st = 0
et = 2

cmaps = ['green7', 'orange7', 'blue7']
colors = ['orange7', 'blue7']

# cmaps = ['blue7', 'green7', 'red7']
# colors = ['green7', 'orange7']

legends_0 = []
legends_1 = []

for index,t in enumerate(range(0, 6, 2)):
    s_profile = o3_profile.isel(Time=t)
    e_profile = o3_profile.isel(Time=t+1)

    s_t = s_profile.Time.dt.strftime('%H:%M').values
    e_t = e_profile.Time.dt.strftime('%H:%M').values

    s_legend = axs[0].plot(s_profile*1e3, # ppbv
                           z_profile.isel(Time=t),
                           linestyle='-',
                           color=cmaps[index],
                           label=s_t)
    e_legend = axs[0].plot(e_profile*1e3, # ppbv
                           z_profile.isel(Time=t),
                           linestyle='--',
                           color=cmaps[index],
                           label=e_t)

    legends_0.extend([s_legend, e_legend])

axs[0].format(xlim=(0, 200),
              ylim=(0, 16.5),
              yticks=2,
              xlabel='O$_3$ (ppbv)',
              ylabel='Altitude (km)')

axs[0].legend(legends_0, loc='b', frame=False, prop={'size': 13}, labelspacing=1.5)

# ---- subplot 2 ----
# get xlabels
coord_pairs = to_np(cross_o3.coords["xy_loc"])
xticks = np.arange(coord_pairs.shape[0])
xlabels = [pair.latlon_str() for pair in to_np(coord_pairs)]
cross_lat = np.array([pair.lat for pair in coord_pairs])
cross_index = (crop_region[2] <= cross_lat) & (cross_lat <= crop_region[3])
index_boolean = np.array([i for i, x in enumerate(cross_index) if x])
vline_s = index_boolean.min()
vline_e = index_boolean.max()

data = cross_o3*1e3  # ppbv

ax = axs[1]

# plot cloud boundary: qcld+qice >= 0.1 g/kg
ax.contour(cross_qcld, levels=[0.01], color='white')

# plot fill
m = ax.contourf(data, cmap='pyart_dbz_r', vmin=0, vmax=200, extend='max', levels=20)

# plot focused region line
ax.axvline(vline_s, color='blue3', linestyle='--')
ax.axvline(vline_e, color='blue3', linestyle='--')

# addd title
# title = np.datetime_as_string(data.coords['Time'].values, unit='s').replace('T', ' ')
title = data.coords['Time'].dt.strftime('%H:%M (UTC)').values
ax.format(title=title, titlesize=13)

# add ticks
#labels = [item.get_text() for item in ax.get_xticklabels()]
#ax.format(xlocator=xticks[::20], xticklabels=xlabels[::20])

ax.set_xticks(xticks[::20])
ax.set_xticklabels(['(' +s.replace(',', '\n') + ')' for s in xlabels[::20]], fontsize=4) # rotation

# plot wind
xs = np.arange(0, cross_w.shape[-1], 1)
ys = to_np(cross_w.coords['vertical'])
step = 2
q = ax.quiver(xs[::step],
              ys[::step],
              to_np(cross_u[::step, ::step]+cross_v[::step, ::step]*-1),
              to_np(cross_w[::step, ::step]))

qk = ax.quiverkey(q, 1, 1.05, 10, r'$10 m/s}$', labelpos='E')


# add annotation
ax.text(0, -0.05,
        'C',
        transform=ax.transAxes)

ax.text(0.95, -0.05,
        'D',
        transform=ax.transAxes)

ax.format(xlocator='null',
          xlabel='',
          ylabel='Altitude (km)',
          ylim=(0, 16.5),
          yticks=2)
cbar = ax.colorbar(m, loc='r', label='O$_3$ (ppbv)', labelsize=12)
cbar.ax.tick_params(labelsize=10) 

# add tropo line
tropo_list = []
lapseC = 2*units.kelvin/units.km
height = False
for index in range(cross_p.sizes['cross_line_idx']):
    tropo = tropCalc(cross_p.isel(cross_line_idx=index)[10:].values*units.hPa,
                     cross_t.isel(cross_line_idx=index)[10:].values*units.K,
                     lapseC=lapseC,
                     height=height,
                     method="wmo")
    tropo_list.append(pressure_to_height_std(tropo).magnitude)
ax.scatter(range(len(tropo_list)), tropo_list, s=2, color='w')

# ---- subplot 3 ----
NA = 6.022e23
R = 8.314

t0 = '2019-07-25 04:20'
t1 = '2019-07-25 05:00'

advh_o3_mean = (advh_o3* P * NA / (R*T)).where(subset).mean(dim=['south_north', 'west_east'])*1e-21
advz_o3_mean = (advz_o3* P * NA / (R*T)).where(subset).mean(dim=['south_north', 'west_east'])*1e-21
chem_o3_mean = (chem_o3* P * NA / (R*T)).where(subset).mean(dim=['south_north', 'west_east'])*1e-21
net_o3_mean = advh_o3_mean + advz_o3_mean + chem_o3_mean

l0 = axs[2].plot(advh_o3_mean.sel(Time=t1) - advh_o3_mean.sel(Time=t0),
                  z_profile.isel(Time=0),
                  color='gray8',
                  linestyle='dotted',
                  label='advh')
l1 = axs[2].plot(advz_o3_mean.sel(Time=t1) - advz_o3_mean.sel(Time=t0),
                  z_profile.isel(Time=0),
                  color='gray8',
                  linestyle='dashed',
                  label='advz')
twiny = axs[2].twiny()
l2 = twiny.plot((advh_o3_mean+advz_o3_mean).sel(Time=t1) - (advh_o3_mean+advz_o3_mean).sel(Time=t0),
                  z_profile.isel(Time=0),
                  color='orange8',
                  linestyle='dotted',
                  label='advh+advz')
l3 = twiny.plot(chem_o3_mean.sel(Time=t1) - chem_o3_mean.sel(Time=t0),
                  z_profile.isel(Time=0),
                  color='orange8',
                  linestyle='dashed',
                  label='chem')
l4 = twiny.plot(net_o3_mean.sel(Time=t1) - net_o3_mean.sel(Time=t0),
                z_profile.isel(Time=0),
                color='orange8',
                label='net')

twiny.format(xcolor='orange8', xlim=(-10, 10), ylim=(10, 14), ticklabelsize=10)

axs[2].format(
           xcolor='gray8',
           xlim=(-60, 60),
           ylim=(10, 14),
           yticks=1,
           ylabel='Altitude (km)',
           xlabel='O$_3$ Tendency (10$^{11}$ molec. cm$^{-3}$)',
           grid=False,
           title=f'{t0[-5:]} ~ {t1[-5:]} (UTC)',
           titlesize=13
)
axs[2].legend([l0, l1, l2, l3, l4], loc='r', frame=False, prop={'size': 13}, labelspacing=1.5)


# save fig
fig.savefig(f'../figures/tendency_o3_2019_h.png')

### Summation of tendency

Convert ppmv to molec. cm$^{-3}$:

C$_n$ [molec. cm$^{-3}$] = ppp \* P [Pa] \* N$_A$ / (R * T [k]) \*10$^{-6}$

= ppm \* P [hPa] \* N$_A$ / (R * T [k]) \*10$^{-10}$

With LNOx:

In [ ]:
NA = 6.022e23
R = 8.314
bottom = 10 # km
top = 14 # km

advh_o3_sum = (advh_o3* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advh')

advz_o3_sum = (advz_o3* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advz')

total_adv_sum = (advh_o3_sum + advz_o3_sum).rename('advh+advz')

chem_o3_sum = (chem_o3* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('chem')

chem_o3_nolnox_sum = (chem_o3_nolnox* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('chem')

o3dt_sum = (o3* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                .rename('o3dt')

# convective period
t0 = '2019-07-25 04:20'
t1 = '2019-07-25 05:00'

accumulated_advh = advh_o3_sum.sel(Time=t1) - advh_o3_sum.sel(Time=t0)
accumulated_advz = advz_o3_sum.sel(Time=t1) - advz_o3_sum.sel(Time=t0)
accumulated_adv = total_adv_sum.sel(Time=t1) - total_adv_sum.sel(Time=t0)
accumulated_chem = chem_o3_sum.sel(Time=t1) - chem_o3_sum.sel(Time=t0)
accumulated_chem_nolnox = chem_o3_nolnox_sum.sel(Time=t1) - chem_o3_nolnox_sum.sel(Time=t0)
accumulated_o3dt = o3dt_sum.sel(Time=t1) - o3dt_sum.sel(Time=t0)

diff_accumulated_chem = (accumulated_chem - accumulated_chem_nolnox).rename('diff_accumulated_chem')
accumulated_adv_chem = (accumulated_chem + accumulated_adv).rename('chem+adv')

print('Convective period')
display(accumulated_advh, accumulated_advz, accumulated_adv, accumulated_chem, accumulated_adv_chem, accumulated_o3dt, diff_accumulated_chem)

In [ ]:
# life cycle
t0 = '2019-07-25 03:20'
t1 = '2019-07-25 05:40'

accumulated_advh = advh_o3_sum.sel(Time=t1) - advh_o3_sum.sel(Time=t0)
accumulated_advz = advz_o3_sum.sel(Time=t1) - advz_o3_sum.sel(Time=t0)
accumulated_adv = total_adv_sum.sel(Time=t1) - total_adv_sum.sel(Time=t0)
accumulated_chem = chem_o3_sum.sel(Time=t1) - chem_o3_sum.sel(Time=t0)
accumulated_chem_nolnox = chem_o3_nolnox_sum.sel(Time=t1) - chem_o3_nolnox_sum.sel(Time=t0)
accumulated_o3dt = o3dt_sum.sel(Time=t1) - o3dt_sum.sel(Time=t0)

diff_accumulated_chem = (accumulated_chem - accumulated_chem_nolnox).rename('diff_accumulated_chem')
accumulated_adv_chem = (accumulated_chem + accumulated_adv).rename('chem+adv')

print('Life cycle')
display(accumulated_advh, accumulated_advz, accumulated_adv, accumulated_chem, accumulated_adv_chem, accumulated_o3dt, diff_accumulated_chem)

Without LNOx

In [ ]:
NA = 6.022e23
R = 8.314

# advh_o3_h = (advh_o3 * P * NA / (R*T) * 1e-10).where(subset, drop=True).mean(dim=['south_north', 'west_east'])\
#                    .diff(dim='Time').sel(Time=subset_t)

advh_o3_sum = (advh_o3_nolnox* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advh')

advz_o3_sum = (advz_o3_nolnox* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('advz')

total_adv_sum = (advh_o3_sum + advz_o3_sum).rename('advh+advz')

chem_o3_sum_nolnox = (chem_o3_nolnox* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                        .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                        .rename('chem')


o3dt_sum = (o3_nolnox* P * NA / (R*T) * 1e-10).where(subset & (z >= bottom) & (z <= top), drop=True)\
                .mean(dim=['south_north', 'west_east', 'bottom_top'])\
                .rename('o3dt')

# # t0 = '2019-07-25 03:20'
# # t1 = '2019-07-25 05:40'

# t0 = '2019-07-25 04:20'
# t1 = '2019-07-25 05:00'

# accumulated_adv = total_adv_sum.sel(Time=t1) - total_adv_sum.sel(Time=t0)
# accumulated_chem_nolnox = chem_o3_sum.sel(Time=t1) - chem_o3_sum.sel(Time=t0)
# accumulated_o3dt = o3dt_sum.sel(Time=t1) - o3dt_sum.sel(Time=t0)

# diff_accumulated_chem = (accumulated_chem - accumulated_chem_nolnox).rename('diff_accumulated_chem')

# display(accumulated_adv, accumulated_chem, accumulated_o3dt, diff_accumulated_chem)



# convective period
t0 = '2019-07-25 04:20'
t1 = '2019-07-25 05:00'

accumulated_advh = advh_o3_sum.sel(Time=t1) - advh_o3_sum.sel(Time=t0)
accumulated_advz = advz_o3_sum.sel(Time=t1) - advz_o3_sum.sel(Time=t0)
accumulated_adv = total_adv_sum.sel(Time=t1) - total_adv_sum.sel(Time=t0)
accumulated_chem_nolnox = chem_o3_nolnox_sum.sel(Time=t1) - chem_o3_nolnox_sum.sel(Time=t0)
accumulated_o3dt = o3dt_sum.sel(Time=t1) - o3dt_sum.sel(Time=t0)
accumulated_adv_chem = (accumulated_chem_nolnox + accumulated_adv).rename('chem+adv')

print('Convective period')
display(accumulated_advh, accumulated_advz, accumulated_adv, accumulated_chem_nolnox, accumulated_adv_chem, accumulated_o3dt)

In [ ]:
# life cycle
t0 = '2019-07-25 03:20'
t1 = '2019-07-25 05:40'

accumulated_advh = advh_o3_sum.sel(Time=t1) - advh_o3_sum.sel(Time=t0)
accumulated_advz = advz_o3_sum.sel(Time=t1) - advz_o3_sum.sel(Time=t0)
accumulated_adv = total_adv_sum.sel(Time=t1) - total_adv_sum.sel(Time=t0)
accumulated_chem_nolnox = chem_o3_nolnox_sum.sel(Time=t1) - chem_o3_nolnox_sum.sel(Time=t0)
accumulated_o3dt = o3dt_sum.sel(Time=t1) - o3dt_sum.sel(Time=t0)
accumulated_adv_chem = (accumulated_chem_nolnox + accumulated_adv).rename('chem+adv')

print('Life cycle')
display(accumulated_advh, accumulated_advz, accumulated_adv, accumulated_chem_nolnox, accumulated_adv_chem, accumulated_o3dt)

Whole process:

In [ ]:
duration = (total_adv_sum.Time[-1] - total_adv_sum.Time[0]).values / np.timedelta64(1, 's')
accumulated_adv = total_adv_sum[-1] - total_adv_sum[0]
accumulated_chem = chem_o3_sum[-1] - chem_o3_sum[0]
accumulated_chem_nolnox = chem_o3_nolnox_sum[-1] - chem_o3_nolnox_sum[0]
accumulated_o3dt = o3dt_sum[-1] - o3dt_sum[0]

diff_accumulated_chem = (accumulated_chem - accumulated_chem_nolnox).rename('diff_accumulated_chem')

display(accumulated_adv, accumulated_chem, accumulated_o3dt, diff_accumulated_chem)

In [ ]:
fig, axs = plot.subplots(ncols=2, share=0)

l1 = (chem_o3_sum - chem_o3_sum_nolnox).plot(ax=axs[0],
                                             label='with LNO$_x$ - no LNO$_x$',
                                             color='blue6')
l2 = chem_o3_sum.plot(ax=axs[1],
                      label='with LNO$_x$',
                      color='red6')

l3 = chem_o3_sum_nolnox.plot(ax=axs[1],
                      label='without LNO$_x$',
                      color='blue6')

axs.format(grid=False,
           xlabel='Time (UTC)',
           xlim=(chem_o3_sum.Time.values[0], chem_o3_sum.Time.values[-1]),
           xlocator=('minute', range(0, 60, 30)),
           xminorlocator=('minute', range(0, 60, 10)),
           xformatter='%H:%M',
           ylabel='(ppbv)',
           suptitle='Accumulated O$_3$ chemistry tendency (2019-07-25)')
axs[1].legend([l1, l2, l3], ncols=1, loc='r')

In [ ]:
# ((o3_region - o3_region_nolnox)*1e3).sel(bottom_top=slice(10, 25))
((o3_region - o3_region_nolnox)*1e3).sel(Time=slice('2019-07-25 05:30', '2019-07-25 06:00')).plot.line(y="bottom_top", hue="Time")

Is there no need to check the effect of LNOx on O$_3$?

In [ ]:
# # set the interp levels
# bottom = 8 # km
# top = 12 # km

# # interpolate the tendency to levels
# advh_o3_bottom = interplevel(advh_o3, z, bottom)
# advz_o3_bottom = interplevel(advz_o3, z, bottom)
# advh_o3_top = interplevel(advh_o3, z, top)
# advz_o3_top = interplevel(advz_o3, z, top)

# # sum the tendency
# advh_o3_bottom = advh_o3_bottom.where(subset, drop=True)\
#                                 .sum(dim=['south_north', 'west_east'])\
#                                 .rename('advh_o3_bottom')
# advz_o3_bottom = advz_o3_bottom.where(subset, drop=True)\
#                                 .sum(dim=['south_north', 'west_east'])\
#                                 .rename('advz_o3_bottom')
# advh_o3_top = advh_o3_top.where(subset, drop=True)\
#                             .sum(dim=['south_north', 'west_east'])\
#                             .rename('advh_o3_top')
# advz_o3_top = advz_o3_top.where(subset, drop=True)\
#                             .sum(dim=['south_north', 'west_east'])\
#                             .rename('advz_o3_top')

# chem_o3_box = chem_o3.where(subset & (z >= bottom) & (z <= top), drop=True)\
#                         .sum(dim=['south_north', 'west_east', 'bottom_top'])\
#                         .rename('chem_o3')

In [ ]:
# fig, axs = plot.subplots()

# legends = []
# for var in [advh_o3_bottom, advz_o3_bottom, advh_o3_top, advz_o3_top, chem_o3_box]:
#     if 'bottom' in var.name:
#         color = 'blue7'
#     elif 'top' in var.name:
#         color = 'red7'
#     else:
#         color = 'green7'
        
#     if 'advh' in var.name:
#         linestyle = '-'
#     elif 'advz' in var.name:
#         linestyle = '--'
#     else:
#         linestyle = '-'

#     # pptv/s
#     legend = (var.diff(dim='Time')/(var.Time.diff(dim='Time')/np.timedelta64(1, 's')).astype(int)*1e6)\
#               .plot(ax=axs, label=var.name, color=color, linestyle=linestyle)
#     legends.append(legend)

# print(legends)
# axs.legend(legends, ncols=1, loc='r')
# axs.format(ylabel='O$_3$ tendencies (pptv/s)', title='', grid=False)